# Example 2: Extended Kalman Filter

## Contents
* [Acknowledgements](#ackw)
* [Overview](#overview) 
    * [Extended Kalman Filter](#ekf)
    * [Summary EKF](#sumekf)
    * [Motion Model](#motion_model)
    * [Observation Model](#observation_model)
* [Include files](#include_files)
* [The main function](#m_func)
* [Results](#results)
* [Source Code](#source_code)

## <a name="overview"></a> Overview


Notebook <a href="../../exe1/exe.ipynb">Kalman Filter</a> discussed the Kalman filter. This filter is an optimal filtering method for linear systems. In this notebook, we will extended the Kalman filter to systems governed by non-linear dynamics. The resulting filter is known as the Extended Kalman Filter or EKF. The discussion below is only meant to be descriptive and we will not go much into details. There are many excellent references around you can further enhance your understanding. Also, the wikipedia entry is   https://en.wikipedia.org/wiki/Extended_Kalman_filter. 

### <a name="ekf"></a> Extended Kalman Filter

Briefly, the EKF is an improvement over the classic Kalman Filter that can be applied to non-linear systems. The crux of the algorithm  remains the predictor-corrector steps just like in the Kalman Filter. In fact to a large extent the two methods are identical. However, the EKF method involves a linearization step of the non-linear system. 

Thus, in EKF, we linearize the nonlinear system around
the Kalman filter estimate, and the Kalman filter estimate is based on the
linearized system. This is the idea of the extended Kalman filter (EKF), which
was originally proposed by Stanley Schmidt so that the Kalman filter could be
applied to nonlinear spacecraft navigation problems.

In this setting we have the following notation:

- $\mathbf{x}_k$ is the state vector at step $k$
- $\hat{\mathbf{x}}_k$ is the predicted state vector at step $k$
- $\mathbf{u}_k$ is the control signal vector at step $k$
- $\mathbf{z}_k$ is the sensor measurement vector at step $k$
- $f$ is the non-linear function describing the dynamics of the system
- $h$ is the non-linear function describing the measurements that is the modeling of the sensors we use
- $\mathbf{P}_k$ is the covariance matrix at step $k$
- $\hat{\mathbf{P}}_k$ is the predicted covariance matrix at step $k$
- $\mathbf{Q}_k$ is the covariance matrix associated with the control signal at step $k$
- $\mathbf{R}_k$ is the covariance matrix associated with the measurement signal at step $k$
- $\mathbf{K}_k$ is the gain matrix at step k
- $\mathbf{w}_k$ is the error vector associated with the control signal at step k
- $\mathbf{v}_k$ is the error vector associated with the measurement signal at step k

We also have the following Jacobian matrices

$$\mathbf{F}_k = \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}=\mathbf{x}_{k-1}}$$

$$\mathbf{L}_k = \frac{\partial f}{\partial \mathbf{w}}|_{\mathbf{x}=\mathbf{x}_{k-1}}$$

$$\mathbf{H}_k = \frac{\partial h}{\partial \mathbf{x}}|_{\mathbf{x}=\mathbf{x}_{k-1}}$$

$$\mathbf{M}_k = \frac{\partial h}{\partial \mathbf{v}}|_{\mathbf{x}=\mathbf{x}_{k-1}}$$

### <a name="sumekf"></a> Summary of EKF

In summary the EKF algorithm is shown below:

**Predict Step**

$$\hat{\mathbf{x}}_k = f(\mathbf{x}_{k-1}, \mathbf{u}_{k}, \mathbf{0})$$

$$\hat{\mathbf{P}}_k = \mathbf{F}_{k-1}\mathbf{P}_{k-1}\mathbf{F}^{T}_{k-1} + \mathbf{L}_{k-1}\mathbf{Q}_{k-1}\mathbf{L}^{T}_{k-1}$$

**Correct Step**

$$\hat{\mathbf{K}}_k = \hat{\mathbf{P}}_{k}\mathbf{H}_{k}(\mathbf{H}_{k}\hat{\mathbf{P}}_{k}\mathbf{H}_{k}^{T} + \mathbf{M}_{k}\mathbf{R}_{k}\mathbf{M}^{T}_{k})^{-1}$$

$$\mathbf{x}_k = \hat{\mathbf{x}}_k + \mathbf{K}_k(\mathbf{z}_{k} - h(\hat{\mathbf{x}}_k,  \mathbf{0}))$$

$$\mathbf{P}_k = (\mathbf{I} - \mathbf{K}_{k}\mathbf{H}^{T}_{k}) \hat{\mathbf{P}}_{k}$$

### <a name="motion_model"></a> Motion Model

The EKF equations above involve the function $f$ that describes the dynamics of the system. Since we simulate a differential driven vehicle here, we will use the following model.

if the angular velocity $w$ is zero:

$$ x_{k+1} = x_k + (V\Delta t + w_0)*cos(\theta_k  + w_1)$$

$$ y_{k+1} = y_k + (V\Delta t + w_0)*sin(\theta_k  + w_1)$$

if the angular velocity $w$ is non zero:

$$\theta_{k+1} = \theta_k + w*\Delta t + w_1$$

$$x_{k+1} = x_k + (\frac{V}{w} + w_0)*(sin(\theta_{k+1}) - sin(\theta_k))$$

$$y_{k+1} = y_k - (\frac{V}{w} + w_0)*(cos(\theta_{k+1}) - cos(\theta_k))$$

Note that the model aboves assumes that the error is additive which may not be true.

### <a name="observation_model"></a> Observation Model

We also need an observation model. In this example we will assume that the observation model is the identity function that is it simply reflects back the  input given

$$h(\mathbf{x}, \mathbf{v}) = \mathbf{x}$$

## <a name="include_files"></a> Include files

```
#include "cubic_engine/base/cubic_engine_types.h"
#include "cubic_engine/estimation/extended_kalman_filter.h"
#include "kernel/dynamics/diff_drive_dynamics.h"
#include "kernel/base/angle_calculator.h"
#include "kernel/utilities/csv_file_writer.h"
#include "kernel/dynamics/system_state.h"
#include "kernel/maths/constants.h"
#include "kernel/base/unit_converter.h"

#include <cmath>
#include <iostream>
#include <tuple>
```

## <a name="m_func"></a> The main function

```
namespace example
{

using cengine::uint_t;
using cengine::real_t;
using kernel::dynamics::DiffDriveDynamics;
using cengine::ExtendedKalmanFilter;
using cengine::DynMat;
using cengine::DynVec;
using kernel::dynamics::SysState;


class ObservationModel
{

public:

    typedef  DynVec<real_t> input_t;

    ObservationModel();

    // simply return the state
    const DynVec<real_t> evaluate(const DynVec<real_t>& input)const;

    // get the H or M matrix
    const DynMat<real_t>& get_matrix(const std::string& name)const;

private:

    DynMat<real_t> H;
    DynMat<real_t> M;
};

ObservationModel::ObservationModel()
    :
      H(2,3, 0.0),
      M(2, 2, 0.0)
{
    H(0, 0) = 1.0;
    H(1,1) = 1.0;
    M(0,0) = 1.0;
    M(1, 1) = 1.0;

}

const DynVec<real_t>
ObservationModel::evaluate(const DynVec<real_t>& input)const{
    return input;
}

const DynMat<real_t>&
ObservationModel::get_matrix(const std::string& name)const{
    if(name == "H"){
        return H;
    }
    else if(name == "M"){
        return M;
    }

    throw std::logic_error("Invalid matrix name. Name "+name+ " not found");
}

const DynVec<real_t> get_measurement(const SysState<3>& state){
   return DynVec<real_t>({state.get("X"), state.get("Y")});
}


}

int main() {
   
    using namespace example;
    uint_t n_steps = 300;

    auto time = 0.0;
    auto dt = 0.5;

    /// angular velocity
    auto w = 0.0;

    /// linear velocity
    auto vt = 1.0;

    std::array<real_t, 2> motion_control_error;
    motion_control_error[0] = 0.0;
    motion_control_error[1] = 0.0;

    DiffDriveDynamics exact_motion_model;
    exact_motion_model.set_matrix_update_flag(false);
    exact_motion_model.set_time_step(dt);

    DiffDriveDynamics motion_model;

    motion_model.initialize_matrices({1.0, 0.0, motion_control_error});
    motion_model.set_time_step(dt);

    ObservationModel observation;

    ExtendedKalmanFilter<DiffDriveDynamics, ObservationModel> ekf(motion_model, observation);

    DynMat<real_t> R(2, 2, 0.0);
    R(0,0) = 1.0;
    R(1, 1) = 1.0;

    DynMat<real_t> Q(2, 2, 0.0);
    Q(0,0) = 0.001;
    Q(1, 1) = 0.001;

    DynMat<real_t> P(3, 3, 0.0);
    P(0, 0) = 1.0;
    P(1, 1) = 1.0;
    P(2, 2) = 1.0;

    ekf.set_matrix("P", P);
    ekf.set_matrix("R", R);
    ekf.set_matrix("Q", Q);

    kernel::CSVWriter writer("state", kernel::CSVWriter::default_delimiter(), true);
    std::vector<std::string> names{"X", "Y", "X_true", "Y_true"};
    writer.write_column_names(names);

    try{

        uint_t counter=0;
        for(uint_t step=0; step < n_steps; ++step){

            std::cout<<"At step: "<<step<<" time: "<<time<<std::endl;

            if(counter == 50){
              w = kernel::UnitConverter::degrees_to_rad(45.0);
            }
            else if(counter == 100){
               w = kernel::UnitConverter::degrees_to_rad(-45.0);
            }
            else if(counter == 150){
               w = kernel::UnitConverter::degrees_to_rad(-45.0);
            }
            else{
                w = 0.0;
            }


            auto motion_input = std::make_tuple(vt, w, motion_control_error);
            auto& exact_state = exact_motion_model.evaluate(motion_input);

            ekf.predict(motion_input);

            auto& state = motion_model.get_state();
            auto z = get_measurement(state);
            ekf.update(z);

            std::cout<<"Position: "<<ekf.get("X")<<", "<<ekf.get("Y")<<std::endl;
            std::cout<<"Orientation: "<<kernel::UnitConverter::rad_to_degrees(ekf.get("Theta"))<<std::endl;
            std::cout<<"V: "<<vt<<", W: "<<w<<std::endl;

            std::vector<real_t> row(4, 0.0);
            row[0] = exact_state.get("X");
            row[1] = exact_state.get("Y");
            row[2] = state.get("X");
            row[3] = state.get("Y");
            writer.write_row(row);

            time += dt;
            counter++;
        }
    }
    catch(std::runtime_error& e){
        std::cerr<<e.what()<<std::endl;
    }
    catch(std::logic_error& e){
        std::cerr<<e.what()<<std::endl;
    }
    catch(...){
        std::cerr<<"Unknown exception was raised whilst running simulation."<<std::endl;
    }
   
    return 0;
}

```

## <a name="results"></a> Results


```
...
At step: 49 time: 24.5
Position: 12.5, 0
Orientation: 0
V: 1, W: 0
At step: 50 time: 25
Position: 12.7436, 0.0484598
Orientation: 22.5
V: 1, W: 0.785398
At step: 51 time: 25.5
Position: 12.9746, 0.144131
Orientation: 22.5
V: 1, W: 0
At step: 52 time: 26
Position: 13.2056, 0.239802
Orientation: 22.5
V: 1, W: 0
At step: 53 time: 26.5
Position: 13.4365, 0.335472
Orientation: 22.5
V: 1, W: 0
At step: 54 time: 27
Position: 13.6675, 0.431143
Orientation: 22.5
V: 1, W: 0
At step: 55 time: 27.5
Position: 13.8985, 0.526814
Orientation: 22.5
V: 1, W: 0
At step: 56 time: 28
Position: 14.1294, 0.622485
Orientation: 22.5
V: 1, W: 0
At step: 57 time: 28.5
Position: 14.3604, 0.718156
Orientation: 22.5
V: 1, W: 0
At step: 58 time: 29
Position: 14.5914, 0.813827
Orientation: 22.5
V: 1, W: 0
At step: 59 time: 29.5
Position: 14.8224, 0.909498
Orientation: 22.5
...
```

<img src="movie.gif"
     alt="Path view"
     style="float: left; margin-right: 10px;" />

## <a name="source_code"></a> Source Code


<a href="../exe.cpp">exe.cpp</a>